# CREMA-D Emotion Recognition benchmark model using MFCC/FBANK features

In this notebook, we will train different classifiers on ER on the CREMA-D dataset using MFCC/FBANK features.

Moreover, we provide the performance that can acquired by classifying samples by chance.

### Import packages

In [12]:
import os

import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import plot_confusion_matrix, classification_report, balanced_accuracy_score, f1_score, precision_score, recall_score

### Define paths

In [13]:
data_dir = os.path.join("/media", "datastore", "c-matsty-data", "datasets", "CREMA-D")

#### Training data paths

In [14]:
mfcc_train_dir = os.path.join(data_dir, "CREMA-D_Preproc_1_training_MFCC")
mfcc_train_input_path = os.path.join(mfcc_train_dir, "input_data")
mfcc_train_labels_path = os.path.join(mfcc_train_dir, "labels")
mfcc_train_actors_path = os.path.join(mfcc_train_dir, "actors")
fbank_train_dir = os.path.join(data_dir, "CREMA-D_Preproc_1_training_FBANK")
fbank_train_input_path = os.path.join(fbank_train_dir, "input_data")
fbank_train_labels_path = os.path.join(fbank_train_dir, "labels")
fbank_train_actors_path = os.path.join(fbank_train_dir, "actors")

#### Test data paths

In [15]:
mfcc_test_dir = os.path.join(data_dir, "CREMA-D_Preproc_1_test_MFCC")
mfcc_test_input_path = os.path.join(mfcc_test_dir, "input_data")
mfcc_test_labels_path = os.path.join(mfcc_test_dir, "labels")
mfcc_test_actors_path = os.path.join(mfcc_test_dir, "actors")
fbank_test_dir = os.path.join(data_dir, "CREMA-D_Preproc_1_test_FBANK")
fbank_test_input_path = os.path.join(fbank_test_dir, "input_data")
fbank_test_labels_path = os.path.join(fbank_test_dir, "labels")
fbank_test_actors_path = os.path.join(fbank_test_dir, "actors")

### Load data

In [16]:
def load_data(input_path, labels_path, actors_path):
    X = []
    Y = []
    A = []
    for input_file_name in os.listdir(input_path):
        file_name = input_file_name.split(".")[0]
        x = np.load(os.path.join(input_path, input_file_name))
        y = np.load(os.path.join(labels_path, file_name + "_labels.npy"))[..., np.newaxis]
        actors = np.load(os.path.join(actors_path, file_name + "_actors.npy"))[..., np.newaxis]
        X.append(x)
        Y.append(y)
        A.append(actors)
    X = np.vstack(X)
    Y = np.vstack(Y)
    A = np.vstack(A)
    return X, Y, A

###### Load training data

In [17]:
X_mfcc_tr, y_mfcc_tr, actors_mfcc_tr = load_data(mfcc_train_input_path, mfcc_train_labels_path, mfcc_train_actors_path)

In [18]:
X_fbank_tr, y_fbank_tr, actors_fbank_tr = load_data(fbank_train_input_path, fbank_train_labels_path, fbank_train_actors_path)

###### Load test data

In [19]:
X_mfcc_ts, y_mfcc_ts, actors_mfcc_ts = load_data(mfcc_test_input_path, mfcc_test_labels_path, mfcc_test_actors_path)

In [20]:
X_fbank_ts, y_fbank_ts, actors_fbank_ts = load_data(fbank_test_input_path, fbank_test_labels_path, fbank_test_actors_path)

In [21]:
label_dict = {value: index  for index, value in enumerate(np.unique(y_mfcc_tr))}

### Prepare data for training

#### Turn labels from strings to integer identifiers

In [22]:
y_mfcc_tr = np.vectorize(label_dict.get)(y_mfcc_tr)
y_fbank_tr = np.vectorize(label_dict.get)(y_fbank_tr)
y_mfcc_ts = np.vectorize(label_dict.get)(y_mfcc_ts)
y_fbank_ts = np.vectorize(label_dict.get)(y_fbank_ts)

#### Reshape input arrays and labels

In [23]:
X_mfcc_tr = X_mfcc_tr.reshape((X_mfcc_tr.shape[0], X_mfcc_tr.shape[1] * X_mfcc_tr.shape[2]))
X_mfcc_ts = X_mfcc_ts.reshape((X_mfcc_ts.shape[0], X_mfcc_ts.shape[1] * X_mfcc_ts.shape[2]))
X_fbank_tr = X_fbank_tr.reshape((X_fbank_tr.shape[0], X_fbank_tr.shape[1] * X_fbank_tr.shape[2]))
X_fbank_ts = X_fbank_ts.reshape((X_fbank_ts.shape[0], X_fbank_ts.shape[1] * X_fbank_ts.shape[2]))

In [24]:
y_mfcc_tr = y_mfcc_tr.flatten()
y_mfcc_ts = y_mfcc_ts.flatten()
y_fbank_tr = y_fbank_tr.flatten()
y_fbank_ts = y_fbank_ts.flatten()

### Normalize data

In [25]:
mfcc_mean = X_mfcc_tr.mean()
mfcc_std = X_mfcc_tr.std()
X_mfcc_tr = (X_mfcc_tr - mfcc_mean) / mfcc_std 
X_mfcc_ts = (X_mfcc_ts - mfcc_mean) / mfcc_std

In [26]:
fbank_mean = X_fbank_tr.mean()
fbank_std = X_fbank_tr.std()
X_fbank_tr = (X_fbank_tr - fbank_mean) / fbank_std 
X_fbank_ts = (X_fbank_ts - fbank_mean) / fbank_std

### Define sample weights

In [27]:
class_counts = [len(y_mfcc_tr[y_mfcc_tr == i]) for i in label_dict.values()]
class_weights = [max(class_counts)/class_count for class_count in class_counts]
class_weight_dict = {class_idx: class_weight for class_idx, class_weight in zip(label_dict.values(), class_weights)}

In [28]:
label_dict

{'ANG': 0, 'DIS': 1, 'FEA': 2, 'HAP': 3, 'NEU': 4, 'SAD': 5}

### Train and test logistic regression classifier

##### Train with MFCC features

In [29]:
multi_class = 'multinomial'
model = LogisticRegression(multi_class=multi_class, max_iter=15000, random_state=0)

In [30]:
sample_weight = [class_weight_dict[label] for label in y_mfcc_tr]

In [31]:
model = model.fit(X_mfcc_tr, y_mfcc_tr, sample_weight=sample_weight)

In [32]:
y_preds = model.predict(X_mfcc_ts)
print(classification_report(y_mfcc_ts, y_preds))
print(balanced_accuracy_score(y_mfcc_ts, y_preds))

              precision    recall  f1-score   support

           0       0.44      0.50      0.47       125
           1       0.26      0.26      0.26       125
           2       0.24      0.25      0.25       125
           3       0.25      0.25      0.25       125
           4       0.40      0.37      0.39       107
           5       0.39      0.36      0.38       125

    accuracy                           0.33       732
   macro avg       0.33      0.33      0.33       732
weighted avg       0.33      0.33      0.33       732

0.33163862928348914


##### Train with FBANK features

In [33]:
model = LogisticRegression(multi_class=multi_class, max_iter=15000, random_state=0)

In [34]:
model = model.fit(X_fbank_tr, y_fbank_tr, sample_weight=sample_weight)

In [35]:
y_preds = model.predict(X_fbank_ts)
print(classification_report(y_fbank_ts, y_preds))
print(balanced_accuracy_score(y_fbank_ts, y_preds))

              precision    recall  f1-score   support

           0       0.54      0.53      0.53       125
           1       0.31      0.24      0.27       125
           2       0.35      0.22      0.27       125
           3       0.35      0.29      0.32       125
           4       0.45      0.53      0.49       107
           5       0.45      0.73      0.55       125

    accuracy                           0.42       732
   macro avg       0.41      0.42      0.41       732
weighted avg       0.41      0.42      0.40       732

0.4234517133956386


### Train and test RandomForest classifier

##### Train with MFCC features

In [36]:
model = RandomForestClassifier(n_estimators=400, random_state=0)

In [37]:
model = model.fit(X_mfcc_tr, y_mfcc_tr, sample_weight=sample_weight)

In [38]:
y_preds = model.predict(X_mfcc_ts)
print(classification_report(y_mfcc_ts, y_preds))
print(balanced_accuracy_score(y_mfcc_ts, y_preds))

              precision    recall  f1-score   support

           0       0.59      0.83      0.69       125
           1       0.48      0.32      0.38       125
           2       0.75      0.12      0.21       125
           3       0.44      0.54      0.49       125
           4       0.56      0.72      0.63       107
           5       0.51      0.66      0.58       125

    accuracy                           0.53       732
   macro avg       0.56      0.53      0.50       732
weighted avg       0.55      0.53      0.49       732

0.5319376947040498


##### Train with FBANK features

In [39]:
model = RandomForestClassifier(n_estimators=400, random_state=0)

In [40]:
model = model.fit(X_fbank_tr, y_fbank_tr, sample_weight=sample_weight)

In [41]:
y_preds = model.predict(X_fbank_ts)
print(classification_report(y_fbank_ts, y_preds))
print(balanced_accuracy_score(y_fbank_ts, y_preds))

              precision    recall  f1-score   support

           0       0.59      0.72      0.65       125
           1       0.54      0.36      0.43       125
           2       0.52      0.13      0.21       125
           3       0.41      0.55      0.47       125
           4       0.56      0.75      0.64       107
           5       0.57      0.69      0.62       125

    accuracy                           0.53       732
   macro avg       0.53      0.53      0.50       732
weighted avg       0.53      0.53      0.50       732

0.5326105919003116


### Chance level performance

In [42]:
trials = 100
avg_f1 = 0.0
avg_prec = 0.0
avg_rec = 0.0
for i in range(trials):
    y_preds = np.random.choice(np.unique(y_fbank_tr), y_fbank_ts.shape[0])
    avg_f1 += f1_score(y_fbank_ts, y_preds, average='macro')
    avg_prec += precision_score(y_fbank_ts, y_preds, average='macro')
    avg_rec += recall_score(y_fbank_ts, y_preds, average='macro')
avg_f1 /= trials
avg_prec /= trials
avg_rec /= trials
print("Macro f1: {}".format(avg_f1))
print("Macro recall: {}".format(avg_rec))
print("Macro precision: {}".format(avg_prec))

Macro f1: 0.1660242992716058
Macro recall: 0.1664618068535826
Macro precision: 0.16641704769631807
